In [1]:
import pandas as pd
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [2]:
data_dir = "/home/jing/Phd_project/project_GBM/gbm_Scripts/gbm_Scripts_bmra_u251mg/01_outputs_2020/"
#info_dir = data_dir

out_dir = "02_outputs_2020"

os.makedirs(out_dir, exist_ok = True)


# Load Data

In [3]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [4]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_Data_norm_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape

(978, 68)

In [5]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values

# gamma_matrix = pd.read_csv(
#     os.path.join(data_dir, "gamma_annotated.csv"),
#     index_col = 0,
# ).values

pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values

In [6]:
# y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)

y_true = 1 / (1 + inhib_conc_matrix / ic50_matrix)

display(y_true.shape)
y_true

(9, 68)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.66666667e-01, 6.43086817e-01, 9.43396226e-01,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.000000

## Run models

In [7]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000, 10, 10, 10, 100)

In [8]:
a_coeffs_df = pd.DataFrame(a_coeffs, index = modules, columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv(os.path.join(out_dir,'a_coeffs.csv'),index_col=0)
#a_coeffs = a_coeffs_df.values
display(a_coeffs_df.astype(bool).sum(axis='columns'))
display(a_coeffs_df)

CDK2        978
CDK4_6      978
PI3K        978
ERK         978
Estrogen    978
EGFR        978
PDGFR       978
Aurora      978
p53         978
dtype: int64

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
CDK2,-1.391699e-05,-0.000012,-0.000005,0.000017,1.083997e-05,-9.057587e-07,-0.000023,-0.000028,-2.841854e-07,-1.698068e-05,...,-0.000011,-1.478100e-05,-7.741192e-06,0.000009,-0.000003,3.453222e-07,0.000005,5.663337e-06,-0.000003,-0.000019
CDK4_6,-5.895941e-06,-0.000005,-0.000004,0.000038,-1.752460e-05,-1.154117e-05,-0.000011,0.000020,1.194430e-06,-2.261586e-05,...,0.000027,-3.960661e-05,2.995574e-05,-0.000003,0.000010,-8.033056e-06,0.000013,4.200978e-06,0.000019,-0.000009
PI3K,-1.386605e-05,0.000018,0.000002,0.000018,-6.864687e-06,2.568985e-05,-0.000004,-0.000003,-1.025347e-05,1.618068e-05,...,0.000042,-4.854722e-06,-1.916981e-05,-0.000007,0.000014,-3.132287e-05,-0.000003,9.657684e-07,0.000002,-0.000006
ERK,-4.431823e-06,0.000005,-0.000027,-0.000010,-2.357668e-05,1.570859e-05,-0.000019,-0.000005,7.070835e-07,4.371857e-07,...,0.000007,3.697961e-05,1.787631e-05,-0.000026,0.000002,7.504554e-02,-0.000003,1.209530e-05,-0.000004,-0.000021
Estrogen,8.106164e-06,-0.000036,-0.000025,-0.000024,-1.123889e-07,1.950417e-06,0.000014,-0.000012,-5.715049e-06,7.051921e-06,...,-0.000013,4.611589e-06,7.662233e-06,0.000003,0.000010,2.488127e-03,-0.000034,3.099164e-06,0.000024,-0.000004
EGFR,1.412740e-05,-0.000010,-0.000005,-0.000002,4.680887e-06,3.106662e-05,0.000030,0.000007,2.361714e-05,1.484277e-05,...,-0.000026,5.099856e-06,-5.303295e-06,-0.000004,0.000023,6.751158e-06,-0.000019,1.602055e-05,-0.000016,0.000003
PDGFR,-4.193629e-07,-0.000003,0.000009,0.000005,-2.269423e-05,2.313675e-05,0.000022,-0.000025,-3.658344e-06,-1.100118e-05,...,-0.000016,6.070541e-09,3.327630e-07,0.000018,-0.000007,1.850030e-05,0.000031,3.766501e-06,0.000026,-0.000012
Aurora,4.142147e-06,0.000016,0.000006,0.000006,1.081152e-05,-1.309190e-05,0.000025,0.000021,2.465953e-05,-6.496701e-06,...,-0.000006,-1.657454e-05,-1.293642e-06,-0.000003,0.000006,5.846417e-04,-0.000009,-3.869256e-06,-0.000015,-0.000011
p53,-2.918485e-05,-0.000014,0.000024,0.000027,1.479070e-05,1.507894e-05,-0.000005,0.000040,-1.736125e-05,-2.347298e-05,...,-0.000005,-1.814628e-05,-4.530416e-05,-0.000015,-0.000009,7.210046e-06,0.000025,5.921867e-06,0.000003,0.000008


In [9]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [10]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,ASG002_U251MG_24H:B14,ASG002_U251MG_24H:B15,ASG002_U251MG_24H:C13,...,ASG002_U251MG_24H:M24,ASG002_U251MG_24H:N22,ASG002_U251MG_24H:N23,ASG002_U251MG_24H:N24,ASG002_U251MG_24H:O10,ASG002_U251MG_24H:O11,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK2,0.101048,0.092031,-0.011180,0.087682,0.065084,0.091590,0.063799,0.092609,0.045405,0.031242,...,-0.007224,0.029503,0.019165,0.036521,0.073077,-0.001555,0.011468,0.053445,0.097074,-0.138455
CDK4_6,-0.034588,0.036245,0.025466,0.013067,0.022824,-0.015145,0.001511,0.016266,0.024925,0.055222,...,0.070154,-0.021688,0.054562,0.042065,0.013358,0.005930,-0.015422,-1.635673,-1.745908,-1.073442
PI3K,-1.671152,-1.761501,-1.537645,-0.174551,-0.196307,-0.208284,-0.261942,0.007506,-0.172437,-0.143874,...,0.370376,0.102777,-0.024845,0.015664,0.324085,-0.077456,-0.866591,-0.660481,-0.786402,-0.510196
ERK,-1.451296,-1.049523,-0.548311,-0.143077,-0.104596,-0.022933,-0.389746,-0.348772,-0.277734,0.160364,...,0.064283,-0.384334,-0.206127,0.014729,-0.807122,-0.092129,-0.255222,-0.968747,-1.986021,-0.894687
Estrogen,0.035618,0.084747,-0.373749,-0.133320,0.054655,-0.006648,-0.070424,0.032687,0.048627,-0.008743,...,0.018496,-0.259226,-0.042653,-0.044271,-1.657682,-1.387374,-1.384575,-0.316980,0.047511,-0.517700
EGFR,0.250666,0.395559,0.392072,-1.614377,-1.014252,0.369631,-1.613522,0.410030,-1.459887,0.244815,...,-1.669163,-1.957727,-1.465084,-1.304875,0.336743,0.353579,0.509665,-1.899566,0.307060,-1.272544
PDGFR,-0.295805,-0.270717,-0.535213,-0.160155,-0.024668,0.023851,0.347525,0.262103,-0.339739,-0.438132,...,-0.251933,0.014736,-0.430127,-0.399836,-0.709114,0.005942,-0.770148,-1.878563,-1.206355,0.761778
Aurora,0.431648,-0.959191,0.308563,-1.499149,-1.931927,-1.655048,-1.619373,-1.999016,-1.438284,0.247515,...,0.096074,-0.990115,-1.380380,-1.468261,0.535399,-0.231497,0.356256,-1.260173,-1.017660,-1.082805
p53,-0.228942,0.245429,0.094512,-1.595964,-1.572741,-1.505859,-0.776653,-0.279042,0.031126,-1.633360,...,0.394738,0.138466,-0.461514,-1.079353,-0.515582,-0.974651,0.424103,0.151840,0.041004,0.003934


In [11]:
R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,ASG002_U251MG_24H:B14,ASG002_U251MG_24H:B15,ASG002_U251MG_24H:C13,...,ASG002_U251MG_24H:M24,ASG002_U251MG_24H:N22,ASG002_U251MG_24H:N23,ASG002_U251MG_24H:N24,ASG002_U251MG_24H:O10,ASG002_U251MG_24H:O11,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK2,0.101048,0.092031,-0.011180,0.087682,0.065084,0.091590,0.063799,0.092609,0.045405,0.031242,...,-0.007224,0.029503,0.019165,0.036521,0.073077,-0.001555,0.011468,0.053445,0.097074,-0.138455
CDK4_6,-0.034588,0.036245,0.025466,0.013067,0.022824,-0.015145,0.001511,0.016266,0.024925,0.055222,...,0.070154,-0.021688,0.054562,0.042065,0.013358,0.005930,-0.015422,-1.635673,-1.745908,-1.073442
PI3K,-1.671152,-1.761501,-1.537645,-0.174551,-0.196307,-0.208284,-0.261942,0.007506,-0.172437,-0.143874,...,0.370376,0.102777,-0.024845,0.015664,0.324085,-0.077456,-0.866591,-0.660481,-0.786402,-0.510196
ERK,-1.451296,-1.049523,-0.548311,-0.143077,-0.104596,-0.022933,-0.389746,-0.348772,-0.277734,0.160364,...,0.064283,-0.384334,-0.206127,0.014729,-0.807122,-0.092129,-0.255222,-0.968747,-1.986021,-0.894687
Estrogen,0.035618,0.084747,-0.373749,-0.133320,0.054655,-0.006648,-0.070424,0.032687,0.048627,-0.008743,...,0.018496,-0.259226,-0.042653,-0.044271,-1.657682,-1.387374,-1.384575,-0.316980,0.047511,-0.517700
EGFR,0.250666,0.395559,0.392072,-1.614377,-1.014252,0.369631,-1.613522,0.410030,-1.459887,0.244815,...,-1.669163,-1.957727,-1.465084,-1.304875,0.336743,0.353579,0.509665,-1.899566,0.307060,-1.272544
PDGFR,-0.295805,-0.270717,-0.535213,-0.160155,-0.024668,0.023851,0.347525,0.262103,-0.339739,-0.438132,...,-0.251933,0.014736,-0.430127,-0.399836,-0.709114,0.005942,-0.770148,-1.878563,-1.206355,0.761778
Aurora,0.431648,-0.959191,0.308563,-1.499149,-1.931927,-1.655048,-1.619373,-1.999016,-1.438284,0.247515,...,0.096074,-0.990115,-1.380380,-1.468261,0.535399,-0.231497,0.356256,-1.260173,-1.017660,-1.082805
p53,-0.228942,0.245429,0.094512,-1.595964,-1.572741,-1.505859,-0.776653,-0.279042,0.031126,-1.633360,...,0.394738,0.138466,-0.461514,-1.079353,-0.515582,-0.974651,0.424103,0.151840,0.041004,0.003934
